# Дипломный проект по SQL

# Введение

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Моя компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

<b>Цель исследования</b>

Проанализировать базу данных для формирования ценностного предложения для нового продукта

# Шаг 1. Устанавливаем соединение

In [1]:
# импорт нужных библиотек
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp',  # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Шаг 2. Знакомимся с данными

Посмотрим содержание таблиц из БД, к которой мы создали коннектор.

In [3]:
query = """
SELECT *
FROM books
LIMIT 5;
"""
table = pd.io.sql.read_sql(query, con = engine)

display(table)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = """
SELECT *
FROM authors
LIMIT 5;
"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = """
SELECT *
FROM publishers
LIMIT 5;
"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = """
SELECT *
FROM reviews
LIMIT 5;
"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
query = """
SELECT *
FROM ratings
LIMIT 5;
"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


# Шаг 3. Пишем запросы согласно заданиям

# 3.1 Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query = """
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'::date

"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,count
0,819


После 1 января 2000 вышло 819 книг.

# 3.2 Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query = """
SELECT title,
       COUNT(DISTINCT review_id) AS review_cnt,
       ROUND(AVG(rating), 2) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r ON b.book_id = r.book_id
     LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, review_cnt DESC

"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,title,review_cnt,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Нами были получены данные о тысяче книг, которые мы отсортировали по средней оценке и количеству обзоров в порядке убывания. Лидером такого списка является книга A Dirty Job (Grim Reaper #1).

# 3.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = """
WITH publisher_books_cnt AS (
    SELECT publisher,
           COUNT(book_id) AS books_cnt
    FROM publishers AS p INNER JOIN books AS b ON p.publisher_id = b.publisher_id
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY books_cnt DESC
    )
SELECT publisher
FROM publisher_books_cnt
WHERE books_cnt = (SELECT MAX(books_cnt)
                   FROM publisher_books_cnt)

"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,publisher
0,Penguin Books


C помощью временной таблицы мы определили количество книг толще 50 страниц, выпущенное каждым издательством, а основным запросом определили издательство с максимальным количеством таких книг. Это издательство Penguin Books.

# 3.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = """

SELECT author,
       AVG(rating) AS mean_rating
FROM authors AS a
     JOIN books AS b ON a.author_id = b.author_id
     JOIN ratings AS r ON b.book_id = r.book_id

WHERE b.book_id IN (
                  SELECT book_id
                  FROM ratings 
                  GROUP BY book_id
                  HAVING COUNT(rating_id) >= 50
                  ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY mean_rating DESC
   LIMIT 1

"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Мы определили автора книг с самой высокой средней оценкой среди книг, которым поставили 50 и более оценок — это автор J.K. Rowling/Mary GrandPré. Показатель средней оценки равен 4.287097.

# 3.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = """
SELECT ROUND(AVG(review_cnt))
FROM (
       SELECT username, 
              COUNT(review_id) AS review_cnt
       FROM reviews 
       WHERE username IN (
                          SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 50)
       GROUP BY  username
      ) AS rereview_count
"""
table = pd.io.sql.read_sql(query, con = engine)
display(table)

,round
0,24.0


В среднем пользователи поставившие более 50 оценок пишут по 24 обзора. 

# Шаг 4. Итоговый вывод

Цель исследования выполнена: мы проанализировали базу данных для формирования ценностного предложения для нового продукта и выяснили, что:
- После 1 января 2000 вышло 819 книг.
- В данных базы содержатся записи о тысяче книг, среди которых максимальные средние значения количества обзоров и рейтинга принадлежит книге "A Dirty Job (Grim Reaper #1)"
- Издательство, которое выпустило наибольшее число книг толще 50 страниц — это издательство Penguin Books.
- Автор книг с самой высокой средней оценкой среди книг, которым поставили 50 и более оценок — это автор J.K. Rowling/Mary GrandPré. Показатель средней оценки равен 4.287097.
- В среднем пользователи поставившие более 50 оценок пишут по 24 обзора.